# 요약

- 개요
    - 주식의 가격에는 다양한 요소들이 영향을 미치지만 그 중 뉴스에 민감하다고 판단하여 뉴스 기사의 제목을 분석하여 긍정/부정 평가를 한다.
1. 데이터 수집
    - 최근 1일, 정확도 순서로 검색어 입력 시 그에 대한 뉴스 제목 정보를 크롤링한다.
2. 전처리
    - konlpy 라이브러리로 제목에 대해서 형태소 분석을 하고 tokenize를 진행한다.
    - 제목을 수치화한 'sentiment' 값을 얻어내고 데이터는 아래와 같다.
    - feature
        - 크롤링한 기사 제목에 대한 bert 모델을 적용한 수치값 (sentiment)
    - target
        - 긍정인 경우 1, 부정인 경우 0으로 라벨링 설정
3. 모델링
    - 삼성전자, 카카오, 네이버, sk하이닉스, 현대자동차의 기사 제목의 sentiment 값을 구하고 600개 정도의 데이터를 직접 라벨링 함. (계속 추가할 예정)
    - DecisionTree, RandomForestClassifier, LogisticRegression 모델을 학습시켜 0.84, 0.85, 0.86의  accuracy가 나옴.
    
4. 성능 평가
    - 최근 삼성의 1일 뉴스 데이터를 적용해본 결과 실제로 긍정적인 뉴스가 많았고 긍정을 예측함.
    - 우리의 목적은 뉴스에 대한 긍정 평가였지만 실제로 주가의 상승으로 이어짐

# 네이버 뉴스 크롤링
- 검색어 입력 받고 정확도순, 최근 1일 데이터 수집
- 제목 수준에서 긍/부정 판단

In [1]:
# 크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm

# 페이지 입력 (1 페이지당 기사 10개 이하)
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# search : 검색어, pd=4 : 최근 1일, start_page : 몇 페이지
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        # 정확도순(디폴트)으로 1일간의 뉴스(pd=4) 
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(
            start_page)
        
        return url
    else:
        # url 부분에서 정확도순서로 1일 데이터를 분류 가능
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(page)
            #url = "https://search.naver.com/search.naver?where=news&query=%EC%B9%B4%EC%B9%B4%EC%98%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2021.09.10&de=2021.09.15&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210910to20210915&is_sug_officeid=0"+"&start=" + str(page)
            urls.append(url)
        return urls

    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)

# 기사 내용 크롤링 함수
def news_attrs_crawler(articles, attrs):
    attrs_content = []
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select(
        "div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver, 'href')
    return url


#####뉴스크롤링 시작#####

# 테스트 횟수
num = int(input('테스트 횟수 입력 : '))
# 검색어 입력
search = input("검색 키워드 입력 : ")
# 검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ", page, "페이지")
# 검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ", page2, "페이지")

# naver url 생성
url = makeUrl(search, page, page2)

# 뉴스 크롤러 실행
news_titles = []
news_url = []

# 일단 제목 수준으로 진행 (기사 내용은 추후에 적용 여부 판단)
# news_contents = []

news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
# ------------------------------------------------------------------------------------------------------------------
    # 뉴스 본문 가져오기 (일단 구현은 해놓음 but 일단 기사 제목 수준에서 진행)
#     content = news_html.select("div#dic_area")
#     if content == []:
#         content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    #content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
#     content = re.sub(pattern=pattern1, repl='', string=content)
#     pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
#     content = content.replace(pattern2, '')
    
    news_titles.append(title)
#    news_contents.append(content)

    try:
        html_date = news_html.select_one(
            "div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ", (page2 + 1 - page) * 10, '개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
#print("\n[뉴스 내용]")
#print(news_contents)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
#print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

# 데이터 프레임 만들기
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles})
# news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
print("중복 제거 후 행 개수: ", len(news_df))

# 데이터 프레임 저장
now = datetime.datetime.now()
news_df.to_csv(f'csv/{search} 뉴스 제목{num}.csv', encoding='utf-8-sig', index=False)

print(type(news_df['title']))
# print(type(news_df['content']))


테스트 횟수 입력 : 777
검색 키워드 입력 : 삼성전자

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):10

크롤링할 종료 페이지:  10 페이지


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:12<00:00,  3.90it/s]


검색된 기사 갯수: 총  100 개

[뉴스 제목]
['삼성전자, 올림픽의 날 앞두고 운동 캠페인', "삼성전자, 'AI로 에너지 절약하는' 비스포크 신제품 공개", "삼성전자, 전 세계에 '비스포크 라이프' 가전 늘린다", '[특징주] 삼성전자, 3거래일 연속 하락…아슬한 ‘7만전자’', '[단독] 한진만 삼성전자 미주총괄 부사장 "美 칩 인력 부족 심각…해결책 절실"', '삼성전자, 3거래일 연속 하락세…‘7만전자’ 턱걸이[특징주]', "반도체 패권 잡자… 삼성전자·TSMC '기술경쟁' 격화", "현대차에 삼성전자 '최첨단 두뇌' 심는다 [삼성-현대차, 모빌리티 반도체 동맹]", '삼성전자 시스템 반도체가 들어간 현대차를 타본다면', '이재용·정의선 손잡았다…삼성전자 車반도체, 현대차에 탑재', '경계현 삼성전자 사장과 인사하는 김기현 대표', '참석자와 대화하는 경계현 삼성전자 DS부문장', "'삼성전자와 이탈리아 토일렛페이퍼의 만남'", '[스페셜리포트]삼성전자·LGD ‘OLED 동맹’...韓 대형 디스플레이 주도권 되찾는다', "삼성전자, '글로벌 비스포크 라이프 2023' 온라인 행사 개최", "삼성전자만 10조 이상 폭풍매수…외국인 쏠림현상에 증시 '착시'", '삼성전자, 현대차에 차량용칩 공급…인포테인먼트 분야 첫 협력', '美中 반도체戰에 낀 삼성전자…‘7만전자’, 엔비디아 ‘조정’ 여파로 흔들리나 [투자360]', "환경산업기술원, 삼성전자 등과 탄소 데이터베이스 구축 '맞손'", '삼성전자 “서울서 갤럭시 언팩, 전세계에 폴더블폰 원조 각인”', '디아이, 삼성전자와 35억 규모 반도체 검사보드 계약', "'삼성 AP 특화 파트너'... 코아시아, 삼성전자의 현대차 AP 최초 공급 수혜 기대", '디아이, 삼성전자와 35억원 규모 반도체 검사보드 공급 계약', "삼성전자, 프린터 복합기 구매 시 '정품토너' 증정 실시", '[단독]韓, 기술 탈취범 80%가 집유… 美선 ‘경제스파이’ 간주 30년형도', '현대차·삼성·SK, 불황에도 작년 고

In [7]:
news_df

,date,title
0,2021-09-15 20:12:22,"[단독] 김범수 동생, 카카오 '옥상옥 지주사'서 퇴직금만 14억"
1,2021-09-15 06:01:03,'카카오 세상' 꿈꿨지만…택시 90% 완전 장악이 독 됐다
2,2021-09-15 19:52:41,"[단독]국회 정무위, 카카오 김범수 '공정위 국감' 증인대 세운다"
3,2021-09-15 09:29:19,"카카오엔터테인먼트, 경력 개발자 첫 공개채용…최대 세 자릿수 규모"
4,2021-09-15 07:27:02,카카오 김범수 공정위 조사…“고의성 입증시 검찰 고발”
...,...,...
94,2021-09-15 13:00:23,전방위 압박에 카카오 상생방안 발표...규제 피할까?
95,2021-09-15 11:53:18,착잡·당혹…뒤숭숭한 카카오 직원들
96,2021-09-15 11:45:33,"네이버·카카오페이, '소상공인 수수료' 카드사보다 최대 3배 높아"
97,2021-09-15 03:02:37,"백기 든 카카오, 골목상권 손뗀다"
